# Lecture 6: Python for Intrument Control (1.5 Hours) #

### ABSTRACT ###

**TODO**

## Unitful Computing with python-quantities (15 Minutes)##

Before getting to acutally controling instruments, we'll briefly mention how to use Python to perform numerical computations in a *unitful* way. We'll demonstrate using python-quantities, which provides a *quantity* data type to represent arrays with associated units (and is how units are handled in IntrumentKit). 

For now, let's go on and import python-quantities.

In [45]:
import numpy as np
import quantities as pq

We can now create quantities by specifying both a magnitude and a unit string.

In [46]:
velocity = pq.Quantity(2, 'm / s')
velocity

array(2) * m/s

Units can also be applied by arithmetically multiplying them with an array from numpy or just a python list. The following two ways of adding units are equvalent:

In [47]:
vader = np.array([1,2,3]) * pq.N
luke = [1,2,3] * pq.N
print vader 
print luke

[ 1.  2.  3.] N
[ 1.  2.  3.] N


Arithmetic operations on quantities respect units.

In [48]:
acceleration = velocity / pq.second
acceleration

array(2.0) * m/s**2

Importantly, operations on quantities must be correct for dimensions, but may use different units.

In [49]:
pq.Quantity(1, 'meter') + pq.Quantity(1, 'foot')

array(1.3048) * m

Now, you may ask what happens when we use dimensionally incorrect units. Look at the below raised error:

In [50]:
pq.Quantity(1, 'meter') + pq.Quantity(1, 'second')

ValueError: Unable to convert between units of "s" and "m"

This looks quite long, but there is basically two main places to look for what the error was: at the top ``ValueError`` and the bottom: ``ValueError: Unable to convert between units of "s" and "m"``. The text in the middle is just tracking where the fuctions were called from and is often not needed to troubleshoot the problem. We can use python statements ``try:`` and ``except`` to help us make errors more readable.

In [51]:
try:
    pq.Quantity(1, 'meter') + pq.Quantity(1, 'second')
except ValueError as ex:
    print(ex)

Unable to convert between units of "s" and "m"


Quanities also support transforming between different unit systems; such conversions are also checked for consistent dimensions.

In [52]:
acceleration.rescale('ft / minute**2')

array(23622.047244094487) * ft/min**2

In [53]:
try:
    acceleration.rescale('kg')
except ValueError as ex:
    print(ex)

Unable to convert between units of "m/s**2" and "kg"


Finally, python-quantities also provides a range of useful physical constants that we can use to quickly construct quantities. 

In [56]:
Sz = pq.constants.hbar * np.array([[1, 0], [0, -1]]) / 2
H = pq.constants.gamma_p * pq.Quantity(7, 'tesla') * Sz
print(H)

[[ 3.5  0. ]
 [ 0.  -3.5]] gamma_p*(h/(2*pi))*T


To make it easier to compare values, you can use the atribute simplified to convert all the units to the defaults for that dimension. 

In [55]:
print(H.simplified)
(H.simplified / (2 * np.pi * pq.constants.hbar)).simplified.rescale('MHz')

[[  9.87424663e-26   0.00000000e+00]
 [  0.00000000e+00  -9.87424663e-26]] kg*m**2/s**2


array([[ 149.02118732,    0.        ],
       [   0.        , -149.02118732]]) * MHz

## More on class inheritance (15 Minutes) ##

## Abstract device component classes (15 Minutes) ##

- channels, I/O, axes

## Let's build up an Instrument and talk to it! (45 Minutes) ##

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import instruments as ik

In [ ]:
ins = ik.generic_scpi.SCPIInstrument.open_tcpip("localhost",8042)

In [ ]:
ins._file.debug = True

In [ ]:
ins._file._query??

In [ ]:
ins.query("VOLTS?")

In [ ]:
ins.sendcmd("VOLTS 43")

In [ ]:
ins.query("ENABLE?")

In [ ]:
ins.sendcmd("ENABLE OFF")

In [ ]:
ins.query("*IDN?")

| SPCI command              | Example usage                                                      | Options                                                                                 | Description                                           |
|---------------------------|--------------------------------------------------------------------|-----------------------------------------------------------------------------------------|-------------------------------------------------------|
| ``*IDN?``                | Query return:<br>``EPQIS16 Demonstration Instrument``              |                                                                                         | Find out what the name of the instument is.           |
| ``VOLTS? <ch>``          | ``VOLTS? 0``<br>``VOLTS? 1``<br>Query return:<br>``{value in mV}`` | ``<ch> = {0,1}``<br>Default: ``0``                                                      | Check what the ouput voltage is set to  (returns mV). |
| ``VOLTS  <ch> <val>``    | ``VOLTS 0 314``<br>``VOLTS 1 159``                               | ``<ch> = {0,1}``<br>Default: ``0``<br>``<val> = voltage value in mV``<br>Default: ``0`` | Set the voltage output (provided value is mV).        |
| ``ENABLE <ch> <state>``  | ``ENABLE 0 OFF``<br>``ENABLE 1 ON``                              | ``<ch> = {0,1}``<br>Default: ``0``<br>``<state> = {ON,OFF}``<br>Default: ``OFF``        | Toggle the voltage channel output on/off.             |     
| ``ENABLE? <ch>``         | ``ENABLE? 0``<br>``ENABLE? 1``<br> Query return:<br>``{ON, OFF}``  | ``<ch> = {0,1}``<br>Default: ``0``                                                      | Check the ouput status of a voltage channel.          | 

``` bash
C:\Users\Sarah\Documents\GitHub\PythonWorkshop-science\epqis16-demos [master ≡ +0 ~2 -0 !]> epqis16 demo_instrument
Waiting for a connection from InstrumentKit on port 8042...........
InstrumentKit connected, opening instrument!
ERR

EPQIS16 Demonstration Instrument

QThread: Destroyed while thread is still running
Fatal Python error: PyImport_GetModuleDict: no module dictionary!

This application has requested the Runtime to terminate it in an unusual way.
Please contact the application's support team for more information.
```

- like catherine's [tutoral](http://catherineh.github.io/programming/2016/08/15/contributing-a-scientific-instrument-driver-to-instrumentkit.html), is there a real physical device we can use or does a dummy need written

## ~~Unit testing (10 Minutes)~~ ##

- ~~so you dont have a bad day or make anyone else's bad~~

## Lots of options of starting points, don't reinvent the wheel! ## 

- [InstrumentKit](http://instrumentkit.readthedocs.io/en/latest/index.html)
- [Instrumental](http://instrumental-lib.readthedocs.io/en/latest/)
- [QuDi](http://qosvn.physik.uni-ulm.de/trac/qudi)  
- [Python IVI](http://alexforencich.com/wiki/en/python-ivi/readme)